In [14]:
import os
import pandas as pd
import numpy as np
import pycountry
import warnings
import country_converter as coco
import logging
import json

# STANDARD DICT { ISO3 : standard_name }

In [15]:
STANDARD_COUNTRY_DICT = dict(zip())

for i in range(len(list(pycountry.countries))):
    STANDARD_COUNTRY_DICT.update({list(pycountry.countries)[i].alpha_3 : list(pycountry.countries)[i].name})
    try:
        STANDARD_COUNTRY_DICT.update({list(pycountry.countries)[i].alpha_3 : list(pycountry.countries)[i].common_name})
    except:
        pass

len(STANDARD_COUNTRY_DICT)

249

In [16]:
with open('../supplemental/STANDARD_COUNTRY_DICT_ISO3.json', 'w') as fp:
    json.dump(STANDARD_COUNTRY_DICT, fp)

# MAIN COUNTRY DICT
### Create empty dictionary

In [17]:
# main dict
COUNTRY_DICT = dict(zip())

### Create long dict using PYCOUNTRY
- use both "name" and "common_name" of all included countries

In [18]:
for i in range(len(list(pycountry.countries))):
    COUNTRY_DICT.update({ list(pycountry.countries)[i].name : list(pycountry.countries)[i].alpha_3 })
    
    #try common_name
    try:
        COUNTRY_DICT.update({ list(pycountry.countries)[i].common_name : list(pycountry.countries)[i].alpha_3 })
    except: 
        pass
    
    #try official_name
    try:
        COUNTRY_DICT.update({ list(pycountry.countries)[i].official_name : list(pycountry.countries)[i].alpha_3 })
    except: 
        pass
    
    # SORT (1st)
    COUNTRY_DICT = {key: value for key, value in sorted(COUNTRY_DICT.items())}

In [19]:
# COUNTRY_DICT

## Final MANUAL patch to dictionary
### Adding missing and REMOVING duplicates!

### IMPORTANT: There can be repeating duplicates. They are removed from top to bottom. 

In [20]:
# ADDING and replacing
COUNTRY_DICT.update({
    
    # ??
    "Laos":"LAO",
    "ANTIGUA & BARBUDA":"ATG",
    "FRANCE (INCLUDING MONACO)":"FRA",
    "ITALY (INCLUDING SAN MARINO)":"ITA",
    "MACAU SPECIAL ADMINSTRATIVE REGION OF CHINA":"MAC",
    
    #CDIAC
    "NIGER":"NER",
    "CURACAO":"CUW",
    "REPUBLIC OF KOREA":"KOR",
    
    #FAO
    "Republic of Korea": "KOR",
    "China, Taiwan Province of": "TWN",

})

# REMOVING (converting to np.nan)
REMOVE_LIST = [
    
    # BP
    "Other Southern Africa",
    
    #CDIAC
    "NETHERLAND ANTILLES AND ARUBA",
    "EAST & WEST PAKISTAN",
    "FRENCH INDO-CHINA",
    "TANGANYIKA",
    "CZECHOSLOVAKIA",
    "FEDERAL REPUBLIC OF GERMANY",
    "JAPAN (EXCLUDING THE RUYUKU ISLANDS)",
    "GUADELOUPE",
    "KUWAITI OIL FIRES",
    "PENINSULAR MALAYSIA",
    "SARAWAK",
    "MARTINIQUE",
    "SABAH",
    "ST. KITTS-NEVIS-ANGUILLA",
    "RHODESIA-NYASALAND",
    "FORMER PANAMA CANAL ZONE",
    "PACIFIC ISLANDS (PALAU)",
    "FEDERATION OF MALAYA-SINGAPORE",
    "ZANZIBAR",
    "FORMER DEMOCRATIC YEMEN",
    "FORMER YEMEN",
    "UNITED KOREA",
    "RWANDA-URUNDI",
    "SUDAN",
    "REPUBLIC OF SUDAN", # DUNNO WHICH IS WHICH
    
    #EIA
    "Former Czechoslovakia",
    "Kosovo",
    
    #IEA
    "Asia (excl. China)",
    "China (incl. Hong Kong, China)", # read methodology, this is a "continent"
    "Africa",
    "Korea", # ?? 
    
    #GCP
    "Africa",
    
    #FAO
    "Australia and New Zealand",
    "Czechoslovakia",
    "Ethiopia PDR",
    "Sudan (former)",
    "Southern Africa",
    "China, mainland", # hmmm
    "Micronesia",
    "French Guyana",
    "Polynesia",
    
]

REMOVE_DICT = dict(zip(REMOVE_LIST, [np.nan] * len(REMOVE_LIST) ))

COUNTRY_DICT.update(REMOVE_DICT)

# SORT (2nd time!)
COUNTRY_DICT = {key: value for key, value in sorted(COUNTRY_DICT.items())}

# NAME -> CODE (CODE_GENERATOR_ISO3)

In [21]:
main_dict = dict(zip())
def CODE_GENERATOR_ISO3(df, show_missing=False, run_pycountry=True, run_coco=True, run_final_patch=True, show_duplicates_nan=False, normalize_names=False):
    
    # -----------CREATE MAIN DICT-------------------#
    
    country_list = df.name.unique()
    
    # for CDIAC
    if normalize_names == True:
        country_list
    
    # ----------------------------------------------#

    
    # STAGE 1: Use COUNTRY_DICT (which uses all pycountry keys and user PATCH of countries to add)
    for i, j in enumerate(country_list):
        try:
            main_dict.update({ j : COUNTRY_DICT[j] })
        except:
            main_dict.update({ j : np.nan })

    # STAGE 2.1: Use pycountry "search_fuzzy()"
    if run_pycountry == True:
        
        for i, j in enumerate(country_list):
            if type(main_dict[j]) == float: #only NaN            
                try:
                    main_dict.update({ j : pycountry.countries.search_fuzzy(j)[0].alpha_3 })
                except:
                    main_dict.update({ j : np.nan })

    # STAGE 2.2: Use coco "convert"
    logging.disable(logging.WARNING) #friggin logging
    if run_coco == True:
        for i, j in enumerate(country_list):
            if type(main_dict[j]) == float: #only NaN
                try:
                    main_dict.update({ j : coco.convert(names=j, to='ISO3', not_found=np.nan) })
                    # print("converted: ", j, " : ", coco.convert(names=j, to='ISO3', not_found=np.nan, enforce_list=False))
                except:
                    main_dict.update({ j : np.nan })
                    warnings.filterwarnings("ignore",message="More")

            # double code fix (choses first in list)
            if type(main_dict[j]) == list:
                main_dict.update({ j : main_dict[j][0] })
                print(f">>> {j} made a list of ({coco.convert(names=j, to='ISO3', not_found=np.nan)}) but is now {main_dict[j]} (first item)\n")
    logging.disable(logging.NOTSET) #friggin logging

                
    # STAGE 3: PATCH (RE-update dictionary (2nd time!) to remove problems generated in stage 2)
    if run_final_patch == True:

        for i, j in enumerate(country_list):
            try:
                main_dict.update({ j : COUNTRY_DICT[j] })
            except:
                pass
        print("PATCH APPLIED")

    
    # ----------------------------------------------#
       
    
    # Missing codes
    missing = []
    for i, j in enumerate(country_list):
        if type(main_dict[j]) == float:
            missing.append(j) 
    if show_missing == True:
        print(f">>> {len(missing)} Missing codes (NaN) in this df\n" , missing)
        
    
    # Add codes to df
    df.code = df.name.map(main_dict)
    
    # Convert duplicates "0" to np.nan
    # df.code = df.code.replace({0:np.nan})    
    
    #Show duplicates and NaN
    if show_duplicates_nan == True:
        df_dup_finder = df.groupby(["code","name"]).count().reset_index().iloc[:,:2]
        dup_list = df_dup_finder[df_dup_finder.duplicated(subset=["code"])].code
        df_dup = df_dup_finder[df_dup_finder.code.isin(dup_list)].groupby(["code","name"]).count()
        print(f"\n>>> {len(df_dup)} DUPLICATES:")
        display(df_dup)
    
    # UPDATE country dict and sort (3rd time!)
    COUNTRY_DICT.update(main_dict)
    return df, main_dict, COUNTRY_DICT

# ----------------------------------------------#
# ----------------------------------------------#
# ----------------------------------------------#


# TEST  - use online link to avoid module importing errors!
# df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")
# df = df.rename(columns={"country":"name"})
# df.insert(0,"code", np.nan)

# CODE_GENERATOR_ISO3(df, show_missing=True, run_pycountry=True, run_coco=True, show_duplicates_nan=True, run_final_patch=True)
# df

# CODE -> NAME (COUNTRY_NAME_GENERATOR_FROM_ISO3)

In [22]:
codes_dict = dict(zip())
def COUNTRY_NAME_GENERATOR_FROM_ISO3(df, show_missing = False, show_duplicates_nan = False):
    
    codes_list = df.groupby("code").count().index
    
    # creates dictionary of {CODES : NAMES}
    for i, j in enumerate(codes_list):
        try:
            codes_dict.update({j : pycountry.countries.search_fuzzy(j)[0].name})
        except:
            codes_dict.update({j : np.nan})
    
    # Add codes to df
    df.name = df.code.map(codes_dict)

    # show missing
    if show_missing == True:
        # find missing names (instead of missing codes due to inverse)
        missing_list = df.groupby(["code", "name"],dropna=False).count().reset_index()
        missing_list = missing_list[missing_list.name.isna()].code
        print("MISSING names:\n" , str(list(missing_list)))

    #Show duplicates and NaN
    if show_duplicates_nan == True:
        df_dup_finder = df.groupby(["code","name"]).count().reset_index().iloc[:,:2]
        dup_list = df_dup_finder[df_dup_finder.duplicated(subset=["code"])].code
        df_dup = df_dup_finder[df_dup_finder.code.isin(dup_list)].groupby(["code","name"]).count()
        print(f"\n>>> {len(df_dup)} DUPLICATES:")
        
        
# ----------------------------------------------#
# ----------------------------------------------#
# ----------------------------------------------#


# TEST  - use online link to avoid module importing errors!
# df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv")
# df = df.rename(columns={"iso_code":"code"})
# df.insert(0,"name", np.nan)

# COUNTRY_NAME_GENERATOR_FROM_ISO3(df, show_missing = True, show_duplicates_nan = True)
# df

In [23]:
print("Imported COUNTRY_DICT & CODE_GENERATOR_ISO3")

Imported COUNTRY_DICT & CODE_GENERATOR_ISO3
